In [1]:
from finviz.screener import Screener
! pip install nest_asyncio
import nest_asyncio
import finviz
import pandas_datareader as pdr
import yfinance as yf
import datetime
import stockquotes
import pandas as pd
import tqdm

In [2]:
nest_asyncio.apply()

In [3]:
filters = ['fa_eps5years_o20', 'fa_epsqoq_o20',
           'fa_epsyoy_o20','fa_sales5years_o20','fa_salesqoq_o20',
           'sh_curvol_o200'] 
stock_list = Screener(filters=filters, table='Performance', order='price')  

In [4]:
print(stock_list)

No. | Ticker | Perf Week | Perf Month | Perf Quart | Perf Half | Perf Year | Perf YTD | Volatility W | Volatility M | Recom | Avg Volume | Rel Volume | Price   | Change | Volume    
--- | ------ | --------- | ---------- | ---------- | --------- | --------- | -------- | ------------ | ------------ | ----- | ---------- | ---------- | ------- | ------ | ----------
1   | VTVT   | 17.00%    | 7.04%      | -6.77%     | 52.11%    | 57.49%    | 55.38%   | 7.84%        | 10.69%       | 2.00  | 3.49M      | 0.65       | 2.89    | 4.33%  | 2,274,688 
2   | OPK    | 2.65%     | -17.29%    | 4.27%      | 46.37%    | 172.94%   | 17.47%   | 5.90%        | 6.91%        | 1.80  | 8.40M      | 0.59       | 4.64    | -1.69% | 4,981,445 
3   | SIGA   | -7.55%    | 3.12%      | -5.03%     | -6.51%    | 65.66%    | -9.08%   | 4.55%        | 3.98%        | -     | 240.29K    | 1.10       | 6.61    | -0.90% | 265,285   
4   | CTMX   | 3.41%     | -5.67%     | 12.01%     | 18.25%    | 135.18%   | 29.62%   | 4.

In [11]:
def moving_average(yahoo_df, days):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=days)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].mean())

def week52_low_high(yahoo_df):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=365)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].max()), float(df_days['Close'].min())


screened_stocks = {}
for stock in tqdm.tqdm(stock_list):
    screened_stocks[stock['Ticker']] = {}
    
    finviz_stats = finviz.get_stock(stock['Ticker'])
    try:
        yahoo_df = pdr.get_data_yahoo(stock['Ticker'], interval = "d")
    except:
        continue
    SMA200_value = moving_average(yahoo_df, days=200)
    SMA150_value = moving_average(yahoo_df, days=150)
    SMA50_value = moving_average(yahoo_df, days=50)
    SMA200_percent = float(finviz_stats['SMA200'].replace("%",""))
    SMA50_percent = float(finviz_stats['SMA50'].replace("%",""))
    prev_close = float(finviz_stats['Prev Close'].replace("$",""))
    week52_high, week52_low = week52_low_high(yahoo_df)
    
    screened_stocks[stock['Ticker']]['SMA200_value'] = SMA200_value
    screened_stocks[stock['Ticker']]['SMA150_value'] = SMA150_value
    screened_stocks[stock['Ticker']]['SMA50_value'] = SMA50_value
    screened_stocks[stock['Ticker']]['SMA200_percent'] = SMA200_percent
    screened_stocks[stock['Ticker']]['SMA50_percent'] = SMA50_percent
    screened_stocks[stock['Ticker']]['prev_close'] = prev_close
    screened_stocks[stock['Ticker']]['week52_high'] = week52_high
    screened_stocks[stock['Ticker']]['week52_low'] = week52_low
    
    # Positive 200d MA positive
    if SMA200_percent > 0:
        SMA200_slope_rule = True
    else:
        SMA200_slope_rule = False
    screened_stocks[stock['Ticker']]['SMA200_slope_rule'] = SMA200_slope_rule
        
    # 150d MA greater than 200d MA
    if SMA150_value > SMA200_value:
        SMA150_greater_SMA200_rule = True
    else:
        SMA150_greater_SMA200_rule = False
    screened_stocks[stock['Ticker']]['SMA150_greater_SMA200_rule'] = SMA150_greater_SMA200_rule
        
    # 50d MA greater than 150d MA
    if SMA50_value > SMA150_value:
        SMA50_greater_SMA150_rule = True
    else:
        SMA50_greater_SMA150_rule = False
    screened_stocks[stock['Ticker']]['SMA50_greater_SMA150_rule'] = SMA50_greater_SMA150_rule
        
    # Close above 50d MA
    if prev_close > SMA50_value:
        close_greater_SMA50_rule = True
    else:
        close_greater_SMA50_rule = False
    screened_stocks[stock['Ticker']]['close_greater_SMA50_rule'] = close_greater_SMA50_rule
        
    # 52 week high low span rule
    if 0.75*week52_high > 1.25*week52_low:
        week52_span_rule = True
    else:
        week52_span_rule = False
    screened_stocks[stock['Ticker']]['week52_span_rule'] = week52_span_rule
    
    # Close above 52 week high - 25%
    if prev_close > 0.75*week52_high:
        close_above_52weekhigh_rule = True
    else:
        close_above_52weekhigh_rule = False
    screened_stocks[stock['Ticker']]['close_above_52weekhigh_rule'] = close_above_52weekhigh_rule

output_list = []
for stock in screened_stocks.keys():
    cols = ["Ticker"] + list(screened_stocks[stock].keys())
    temp_list = []
    temp_list.append(stock)
    for rule in screened_stocks[stock].keys():
        temp_list.append(screened_stocks[stock][rule])
    output_list.append(temp_list)
        
df_out = pd.DataFrame(output_list,columns=cols)

    


100%|██████████| 45/45 [01:28<00:00,  1.97s/it]


In [12]:
df_out.head()

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,prev_close,week52_high,week52_low,SMA200_slope_rule,SMA150_greater_SMA200_rule,SMA50_greater_SMA150_rule,close_greater_SMA50_rule,week52_span_rule,close_above_52weekhigh_rule
0,VTVT,2.085255,2.166176,2.506389,30.86,21.78,2.77,3.19,1.49,True,True,True,True,True,True
1,OPK,4.247810,4.488039,4.984444,11.35,-3.58,4.72,5.99,1.15,True,True,True,False,True,True
2,SIGA,6.940584,6.946765,6.599722,-2.49,-3.15,6.67,7.76,3.99,False,True,False,True,True,True
3,CTMX,7.376971,7.547255,8.074444,12.63,8.12,8.76,14.64,3.61,True,True,True,True,True,False
4,RBBN,6.169489,6.900588,8.820555,54.52,4.12,8.65,11.14,2.03,True,True,True,False,True,True


In [13]:
df_out.to_csv("screened_stocks.csv")